In [ ]:
import random as ran
from math import log
import matplotlib.pyplot as plt
from seaborn import histplot
import pandas as pd

In [ ]:
alphaCounter=[6,10,7,5]
lamdaCounter=[0.3,0.2,0.7,1.0]

In [ ]:
def generateExpo(lamda):
  return -1/lamda*log(ran.uniform(0,1))

In [ ]:
def generateGamma(alpha,lamda):
  generatedData=0
  for i in range(alpha):
    generatedData=generatedData+generateExpo(lamda)
  return generatedData

In [ ]:
class people():
  def __init__(self,n,Xn,An):
    self.n=n
    self.Xn=Xn
    self.An=An
    # self.counter=-1
    self.Sn=0
    self.SSn=0
    self.Dn=0

In [ ]:
class Counter():
  def __init__(self,i):
    self.alpha=alphaCounter[i]
    self.lamda=lamdaCounter[i]
    self.cus=None
    self.queue=[]
    self.remainingSn=0
    self.P=[]

  def isEnteredByCus(self,pn,SSn):
    self.cus=pn
    self.remainingSn=generateGamma(self.alpha,self.lamda)
    self.cus.Sn=self.remainingSn
    self.cus.SSn=SSn

  def updateRemainingSn(self,timeInterval):
    self.remainingSn=self.remainingSn-timeInterval

  def moveToEmptyCounter(self,perviousTimePoint):
    self.preXn=self.remainingSn
    self.lastTimePoint=perviousTimePoint+self.preXn
    self.cus.Dn=self.lastTimePoint
    self.P.append(self.cus)
    self.cus=None
    self.remainingSn=0
    return self.lastTimePoint,self.preXn

  def haveAddedCusToQueue(self,pn):
    self.queue.append(pn)

  def nextCus(self,SSn):
    self.cus=self.queue.pop(0)
    self.remainingSn=generateGamma(self.alpha,self.lamda)
    self.cus.Sn=self.remainingSn
    self.cus.SSn=SSn

  def CanAddCusNowToCounter(self):
    return len(self.queue)>0 and self.remainingSn<0

  def logOfCounterStatus(self,an,preXn,checkQ,arrivedCus=-1,
                         actionText=""):
    if checkQ :
      print(">------------ Between time",preXn,"------------<")
    else:
      print("<---------- Intervarival time",preXn,"--------->")

    print("At time:",an,end=" -> ")

    if checkQ :
      print("Cus",self.cus.n,"enter counter")
    else:
      print("Cus",arrivedCus,actionText)
    print("Now Cus:",self.cus.n)
    print("Remaining Sn:",self.remainingSn)
    print("Queue:",[x.n for x in self.queue])
    print()

# Forming Function

In [ ]:
def generateExperimentOutputAtCounter(i,timeBound,wantEmptyQ=True,showProcess=False):
  counterNow=Counter(i)

  n=0
  xn=0
  an=0

  pn=people(n,0,an)
  counterNow.isEnteredByCus(pn,an)
  if showProcess:counterNow.logOfCounterStatus(an,xn,False,pn.n,
                                "arrive and enter to counter")

  while(an<timeBound):
    n=n+1
    xn=generateExpo(0.25)
    an=an+xn

    pn=people(n,xn,an)
    counterNow.updateRemainingSn(xn)

    preAn=an-xn
    preXn=0
    while(counterNow.CanAddCusNowToCounter()):
      counterNow.updateRemainingSn(preXn-xn)

      preAn,preXn=counterNow.moveToEmptyCounter(preAn)
      # preXn=preAn-(an-xn)
      counterNow.nextCus(preAn)
      if showProcess:counterNow.logOfCounterStatus(preAn,preXn,True)
      counterNow.updateRemainingSn(xn-preXn)

    if(counterNow.CanAddCusNowToCounter()):
      counterNow.isEnteredByCus(pn,an)
      if showProcess:counterNow.logOfCounterStatus(an,xn,False,pn.n,
                                    "have arrived and enter to counter")
    else:
      counterNow.haveAddedCusToQueue(pn)
      if showProcess:counterNow.logOfCounterStatus(an,xn,False,pn.n,
                                    "have arrived and stay in queue")

  if showProcess:
    print()
    print("--------------------------No New Cus---------------------------")
    print()

  if wantEmptyQ:
    an,xn=counterNow.moveToEmptyCounter(an)

    while(len(counterNow.queue)>0):
      counterNow.nextCus(an)
      if showProcess:counterNow.logOfCounterStatus(an,xn,True)
      an,xn=counterNow.moveToEmptyCounter(an)

  return counterNow.P

# Test Function

In [ ]:
pTest=generateExperimentOutputAtCounter(2,60,showProcess=True)

<---------- Intervarival time 0 --------->
At time: 0 -> Cus 0 arrive and enter to counter
Now Cus: 0
Remaining Sn: 13.597555191548144
Queue: []

<---------- Intervarival time 0.5515168237995746 --------->
At time: 0.5515168237995746 -> Cus 1 have arrived and stay in queue
Now Cus: 0
Remaining Sn: 13.04603836774857
Queue: [1]

<---------- Intervarival time 5.027759688051615 --------->
At time: 5.57927651185119 -> Cus 2 have arrived and stay in queue
Now Cus: 0
Remaining Sn: 8.018278679696955
Queue: [1, 2]

>------------ Between time 8.018278679696955 ------------<
At time: 13.597555191548146 -> Cus 1 enter counter
Now Cus: 1
Remaining Sn: 15.985109298946757
Queue: [2]

<---------- Intervarival time 14.24746216878566 --------->
At time: 19.82673868063685 -> Cus 3 have arrived and stay in queue
Now Cus: 1
Remaining Sn: 9.755925809858052
Queue: [2, 3]

<---------- Intervarival time 5.176835211097515 --------->
At time: 25.003573891734366 -> Cus 4 have arrived and stay in queue
Now Cus: 1


In [ ]:
for i in range(len(pTest)):
  pn=pTest[i]
  print("<---------- intervarrival time",pn.Xn,"--------->")
  print("At time",pn.An)
  print("Customer:",pn.n)
  print("Waiting time in queue:",pn.SSn-pn.An)
  print("Start Service Time:",pn.SSn)
  print("Service Time",pn.Sn)
  print("Depart at",pn.Dn)
  print()

<---------- intervarrival time 0 --------->
At time 0
Customer: 0
Waiting time in queue: 0
Start Service Time: 0
Service Time 13.597555191548144
Depart at 13.597555191548146

<---------- intervarrival time 0.5515168237995746 --------->
At time 0.5515168237995746
Customer: 1
Waiting time in queue: 13.046038367748572
Start Service Time: 13.597555191548146
Service Time 15.985109298946757
Depart at 29.582664490494903

<---------- intervarrival time 5.027759688051615 --------->
At time 5.57927651185119
Customer: 2
Waiting time in queue: 24.003387978643712
Start Service Time: 29.582664490494903
Service Time 5.316993655770132
Depart at 34.89965814626504

<---------- intervarrival time 14.24746216878566 --------->
At time 19.82673868063685
Customer: 3
Waiting time in queue: 15.072919465628189
Start Service Time: 34.89965814626504
Service Time 7.886580050684326
Depart at 42.78623819694936

<---------- intervarrival time 5.176835211097515 --------->
At time 25.003573891734366
Customer: 4
Waiting

# Export to xlsx.

In [ ]:
order_customer = [] #คนที่
service_time = [] #service time
start_service_time = [] #start service time
Waiting_time = [] #intervariable time
Depart_at = [] #
At_time = []
P=pTest

for i in range(len(P)):
  order_customer.append(P[i].n)
  service_time.append(P[i].Sn) #service time
  start_service_time.append(P[i].SSn) #start service time
  Waiting_time.append(P[i].SSn-P[i].An)#intervariable time
  Depart_at.append(P[i].Dn) #
  At_time.append(P[i].An)

In [ ]:
df = pd.DataFrame({'order_customer':order_customer,'service_time':service_time,'Waiting_time':Waiting_time,'Depart_at':Depart_at,'At_time':At_time})
df

,order_customer,service_time,Waiting_time,Depart_at,At_time
0,0,13.597555,0.000000,13.597555,0.000000
1,1,15.985109,13.046038,29.582664,0.551517
2,2,5.316994,24.003388,34.899658,5.579277
3,3,7.886580,15.072919,42.786238,19.826739
4,4,8.744076,17.782664,51.530314,25.003574
5,5,9.073165,25.274363,60.603479,26.255951
6,6,6.213433,34.179311,66.816913,26.424168
7,7,8.730035,40.220659,75.546947,26.596253
8,8,8.745964,47.323337,84.292911,28.223611
9,9,5.238037,55.930943,89.530948,28.361968


In [ ]:
from google.colab import files
df
df.to_excel('simulation.xlsx')
files.download('simulation.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Generating Experiment

In [ ]:
allExperiment=[]
for c in range(4):
  eachCounterExperiment=[]
  for i in range(100):
    counterExperiment=generateExperimentOutputAtCounter(c,1440)
    eachCounterExperiment.append(counterExperiment)
  allExperiment.append(eachCounterExperiment)

In [ ]:
len(allExperiment[0][2])

384

In [ ]:
order_customer = [] #คนที่
service_time = [] #service time
start_service_time = [] #start service time
Waiting_time = [] #intervariable time
Depart_at = [] #
At_time = []
P=pTest
sum_each=[]
counter=[]
times=[]

for k in range(4):
  for i in range(len(allExperiment[k])):
    service_timee=[]
    for j in range(len(allExperiment[k][i])):
      counter.append(k+1)
      times.append(i+1)
      allExperiment1=allExperiment[k][i]
      order_customer.append(allExperiment1[j].n+1)
      service_time.append(allExperiment1[j].Sn)
      service_timee.append(allExperiment1[j].Sn) #service time
      start_service_time.append(allExperiment1[j].SSn) #start service time
      Waiting_time.append(allExperiment1[j].SSn-allExperiment1[j].An)#intervariable time
      Depart_at.append(allExperiment1[j].Dn) #
      At_time.append(allExperiment1[j].An)
    sum_each=sum(service_timee)/len(allExperiment[0][i])
    #print(sum_each)

In [ ]:
df = pd.DataFrame({'counter':counter,'times':times,'order_customer':order_customer,'service_time':service_time,'Waiting_time':Waiting_time,'Depart_at':Depart_at,'At_time':At_time})
df

,counter,times,order_customer,service_time,Waiting_time,Depart_at,At_time
0,1,1,1,21.831071,0.000000,21.831071,0.000000
1,1,1,2,22.936392,15.974838,44.767463,5.856232
2,1,1,3,10.481377,36.853419,55.248839,7.914044
3,1,1,4,32.129098,43.757425,87.377938,11.491414
4,1,1,5,9.886548,75.553875,97.264486,11.824063
...,...,...,...,...,...,...,...
144331,4,100,347,2.485495,23.497630,1457.674107,1431.690982
144332,4,100,348,3.623380,21.236091,1461.297486,1436.438016
144333,4,100,349,3.189041,24.208797,1464.486527,1437.088689
144334,4,100,350,5.079250,25.554262,1469.565777,1438.932265


In [ ]:
df.groupby

In [ ]:
from google.colab import files
df
df.to_excel('simulation.xlsx')
files.download('simulation.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>